In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        mess(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
print("this is the running version!")

this is the running version!


### install modules and data folder

In [3]:
import os, sys

- install modules 

In [4]:
!python -m pip install --upgrade --user ortools

In [5]:
import ortools
print(ortools.__version__)

9.9.3963


- install data folder

In [6]:
!git clone https://github.com/likr/kplib.git

fatal: destination path 'kplib' already exists and is not an empty directory.


### utils functions 

In [7]:
'''
fast print log while execute 
'''
def mess_verbose(mes, func_dir=""): 
    print("__verbose__:", func_dir, "@@@", mes, flush=True)

def mess(*some): 
    print(*some, flush=True)

### load data from folder

In [8]:
'''
get all name of files from the respository folder
'''
def get_files_from_folder(folder_path):  
    subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    mess_verbose("Subfolders is", "get_files_from_folder:")
    files_with_paths = []
    for subfolder in subfolders:
        mess(subfolder)
        sub_folder_path = os.path.join(folder_path, subfolder)
        for root, _, files in os.walk(sub_folder_path):
            cnt = 0 
            for file in files:
                file_path = os.path.join(root, file)
                files_with_paths.append(file_path)
                mess(file_path)
                cnt += 1 
                if(cnt >= 1): break 
    return files_with_paths

In [9]:
'''
read file and get convert data in file
'''
def get_data_from_file(file_path): 
    with open(file_path, 'r') as file:
        values, weights = [], [] 
        mess_verbose("get data from file {}".format(file_path), "get_data_from_file:")
        file.readline().split() # ---
        n = int(file.readline().split()[0]) 
        mess("n =", n) 
        C = int(file.readline().split()[0]) 
        mess("C =", C) 
        file.readline().split() # --- 
        for i in range(n):
            p, w = map(int, file.readline().split())
            values.append(p)
            weights.append(w) 
    return C, values, weights 

### solver 

In [10]:
from ortools.algorithms.python import knapsack_solver
import time 

'''
the solver tools for Knapsack 
use branch and bound pattern 
guideline from home page https://developers.google.com/optimization/pack/knapsack
'''
def solve_with_ortools(C, values, weights, time_limit_second=180):
    # Create the solver.
    solver = knapsack_solver.KnapsackSolver(
        knapsack_solver.SolverType.KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER,
        "KnapsackExample",
    )
    # convert to true type of input api  
    weights = [weights]
    capacities = [C]

    solver.init(values, weights, capacities)
    solver.set_time_limit(time_limit_second) 
    start_time = time.time() 
    computed_value = solver.solve()
    wall_time = time.time() - start_time
    
    mess_verbose("solve testcase", "solve_with_ortools:")
    packed_items = []
    packed_weights = []
    total_weight = 0
    mess("Total value =", computed_value)
    for i in range(len(values)):
        if solver.best_solution_contains(i):
            packed_items.append(i)
            packed_weights.append(weights[0][i])
            total_weight += weights[0][i]
    mess("Total weight:", total_weight)
    mess("Number:", len(values))
    mess("Wall time:", wall_time, "second")
    mess("Is optimal:", abs(wall_time - time_limit_second) > 1) 
    mess("Detail:")
    mess("Packed items:", packed_items)
    mess("Packed_weights:", packed_weights)
    return computed_value, total_weight, len(values), wall_time, not (wall_time >= time_limit_second)

In [12]:
C, values, weights = get_data_from_file('/kaggle/working/kplib/06UncorrelatedWithSimilarWeights/n00100/R01000/s099.kp') 

__verbose__: get_data_from_file: @@@ get data from file /kaggle/working/kplib/06UncorrelatedWithSimilarWeights/n00100/R01000/s099.kp
n = 100
C = 4953159


In [13]:
solve_with_ortools(C, values, weights, 180) 

__verbose__: solve_with_ortools: @@@ solve testcase
Total value = 35605
Total weight: 4902732
Number: 100
Wall time: 0.14229822158813477 second
Is optimal: True
Detail:
Packed items: [4, 7, 8, 9, 10, 11, 12, 14, 17, 19, 20, 22, 23, 24, 25, 30, 31, 32, 33, 34, 35, 38, 40, 41, 42, 43, 44, 47, 48, 50, 55, 57, 62, 64, 65, 67, 68, 69, 70, 78, 79, 80, 81, 83, 87, 91, 96, 97, 98]
Packed_weights: [100031, 100022, 100081, 100068, 100025, 100029, 100075, 100044, 100050, 100088, 100079, 100093, 100025, 100089, 100041, 100061, 100099, 100067, 100044, 100038, 100089, 100070, 100019, 100067, 100017, 100064, 100066, 100023, 100086, 100040, 100067, 100016, 100074, 100053, 100062, 100077, 100077, 100020, 100002, 100095, 100059, 100029, 100076, 100080, 100012, 100070, 100027, 100057, 100089]


(35605, 4902732, 100, 0.14229822158813477, True)

### main running

In [14]:
import random as rd 

### pipeline  

In [15]:
'''
summarize pipeline: get files > get data > solver > write results 
'''
def main(folder_path): 
    mess_verbose("--- running ---")
    all_files = get_files_from_folder(folder_path)
    mess("len of all files:", len(all_files))
    limit = 20 
    for file in all_files:
        C, values, weights = get_data_from_file(file)
        res = solve_with_ortools(C, values, weights, 180)
        res = res + (file,)
        # save backup 
        with open(BACKUP_FILE, 'a') as file:
            file.write(str(res) + '\n') 
        limit -= 1 
        if(limit < 0): break
        mess() 

In [17]:
old_stdout = sys.stdout # keep reference to existing stdout
def change_sysout(file_name): 
    global old_stdout
    old_stdout = sys.stdout # keep reference to existing stdout
    sys.stdout = open(file_name, 'w')

In [21]:
rdstr = str(rd.randint(1, 100))
mess(rdstr) 
# change_sysout('log' + rdstr + '.txt')
BACKUP_FILE = 'backup_file' + rdstr + '.txt'
mess(BACKUP_FILE)
main('/kaggle/working/kplib/00Uncorrelated')
sys.stdout = old_stdout

71
backup_file71.txt
__verbose__:  @@@ --- running ---
__verbose__: get_files_from_folder: @@@ Subfolders is
n00200
/kaggle/working/kplib/00Uncorrelated/n00200/R01000/s054.kp
/kaggle/working/kplib/00Uncorrelated/n00200/R10000/s054.kp
n00500
/kaggle/working/kplib/00Uncorrelated/n00500/R01000/s054.kp
/kaggle/working/kplib/00Uncorrelated/n00500/R10000/s054.kp
n05000
/kaggle/working/kplib/00Uncorrelated/n05000/R01000/s054.kp
/kaggle/working/kplib/00Uncorrelated/n05000/R10000/s054.kp
n01000
/kaggle/working/kplib/00Uncorrelated/n01000/R01000/s054.kp
/kaggle/working/kplib/00Uncorrelated/n01000/R10000/s054.kp
n02000
/kaggle/working/kplib/00Uncorrelated/n02000/R01000/s054.kp
/kaggle/working/kplib/00Uncorrelated/n02000/R10000/s054.kp
n00050
/kaggle/working/kplib/00Uncorrelated/n00050/R01000/s054.kp
/kaggle/working/kplib/00Uncorrelated/n00050/R10000/s054.kp
n00100
/kaggle/working/kplib/00Uncorrelated/n00100/R01000/s054.kp
/kaggle/working/kplib/00Uncorrelated/n00100/R10000/s054.kp
n10000
/kaggle/w

In [19]:
# rm -rf